In [10]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import date, time
import datetime as dt
from sqlalchemy import create_engine


In [2]:
from os import listdir
from os.path import isfile, join
myPath = 'Project-3 Data'
files = [f for f in listdir(myPath) if isfile(join(myPath, f))]
files

csvs = [f for f in files if 'csv' in f]
csvs




['1970-1975.csv',
 '1976-1980.csv',
 '1981-1985.csv',
 '1986-1990.csv',
 '1991-1995.csv',
 '1996-2000.csv',
 '2001-2005.csv',
 '2006-2010.csv',
 '2011-2015.csv',
 '2016-2020.csv']

In [25]:


csv_file = f"Project-3 Data/{csvs[0]}" 
df = pd.read_csv(csv_file)
print(f"Loading {csv_file} - {len(df)} records")

for csv in csvs[1:]:
    
    csv_file = f"Project-3 Data/{csv}" 
    current_df = pd.read_csv(csv_file)
    print(f"Loading {csv_file} - {len(current_df)} records")
    df = df.append(current_df, ignore_index=True)
    
df.info()
    

Loading Project-3 Data/1970-1975.csv - 5737 records
Loading Project-3 Data/1976-1980.csv - 8216 records
Loading Project-3 Data/1981-1985.csv - 9496 records
Loading Project-3 Data/1986-1990.csv - 10726 records
Loading Project-3 Data/1991-1995.csv - 13186 records
Loading Project-3 Data/1996-2000.csv - 8080 records
Loading Project-3 Data/2001-2005.csv - 5736 records
Loading Project-3 Data/2006-2010.csv - 5986 records
Loading Project-3 Data/2011-2015.csv - 5003 records
Loading Project-3 Data/2016-2020.csv - 8221 records
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80387 entries, 0 to 80386
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   time             80387 non-null  object 
 1   latitude         80387 non-null  float64
 2   longitude        80387 non-null  float64
 3   depth            80382 non-null  float64
 4   mag              80387 non-null  float64
 5   magType          80386 non-null  object 
 

AttributeError: 'Series' object has no attribute 'split'

In [ ]:
df.to_csv('Database\combined.csv')

In [32]:
engine = create_engine("sqlite:///Database/earthquake_db.db")

In [33]:
df.to_sql(name = "quakes", con=engine, if_exists='replace', index=False)


In [34]:

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, distinct, and_, inspect


Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [35]:
Base.classes.keys()

[]

In [39]:
proper_df = pd.read_csv('Database/combined_ready.csv')
proper_df.dtypes

date                object
time                object
timestamp          float64
latitude           float64
longitude          float64
depth              float64
mag                float64
magType             object
nst                float64
gap                float64
dmin               float64
rms                float64
net                 object
id                  object
updated             object
place               object
type                object
horizontalError    float64
depthError         float64
magError           float64
magNst             float64
status              object
locationSource      object
magSource           object
dtype: object

In [40]:
# proper_df = proper_df.astype({'timestamp':'datetime64[ns]'})
date_time = pd.TimedeltaIndex(proper_df['timestamp'], unit='d') + dt.datetime(1899, 12, 30)
proper_df.insert(3, 'date_time', date_time)                                                                              
proper_df.dtypes

date                       object
time                       object
timestamp                 float64
date_time          datetime64[ns]
latitude                  float64
longitude                 float64
depth                     float64
mag                       float64
magType                    object
nst                       float64
gap                       float64
dmin                      float64
rms                       float64
net                        object
id                         object
updated                    object
place                      object
type                       object
horizontalError           float64
depthError                float64
magError                  float64
magNst                    float64
status                     object
locationSource             object
magSource                  object
dtype: object

In [41]:
proper_df.head()

,date,time,timestamp,date_time,latitude,longitude,depth,mag,magType,nst,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,12/31/1975,34:21.7,27759.60720,1975-12-31 14:34:22.080,36.813500,-121.132333,5.881,2.71,md,50.0,...,2016-12-15T05:09:40.840Z,Central California,earthquake,0.23,0.640,0.09,15.0,reviewed,nc,nc
1,12/31/1975,53:55.9,27759.12079,1975-12-31 02:53:56.256,34.279000,-116.337000,2.330,2.70,md,0.0,...,2016-01-29T21:39:32.720Z,"16km N of Joshua Tree, CA",earthquake,NaN,0.847,NaN,0.0,reviewed,ci,ci
2,12/30/1975,37:52.8,27758.98464,1975-12-30 23:37:52.896,36.905667,-121.637167,-0.155,2.67,md,13.0,...,2016-12-15T05:08:54.636Z,Central California,quarry blast,0.34,1.060,0.77,3.0,reviewed,nc,nc
3,12/30/1975,04:47.5,27758.96166,1975-12-30 23:04:47.424,36.838667,-121.569833,4.774,2.67,md,46.0,...,2016-12-15T05:08:50.998Z,Central California,earthquake,0.21,0.360,0.28,11.0,reviewed,nc,nc
4,12/30/1975,10:27.8,27758.75727,1975-12-30 18:10:28.128,35.781000,-121.421000,11.895,3.15,md,17.0,...,2016-12-15T05:08:39.866Z,offshore Central California,earthquake,1.04,0.610,0.55,4.0,reviewed,nc,nc


In [42]:
proper_df.drop(['date', 'time', 'timestamp'], axis = 1, inplace= True)

proper_df.head()

,date_time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,1975-12-31 14:34:22.080,36.813500,-121.132333,5.881,2.71,md,50.0,97.0,0.06757,0.070,...,2016-12-15T05:09:40.840Z,Central California,earthquake,0.23,0.640,0.09,15.0,reviewed,nc,nc
1,1975-12-31 02:53:56.256,34.279000,-116.337000,2.330,2.70,md,0.0,71.0,NaN,0.187,...,2016-01-29T21:39:32.720Z,"16km N of Joshua Tree, CA",earthquake,NaN,0.847,NaN,0.0,reviewed,ci,ci
2,1975-12-30 23:37:52.896,36.905667,-121.637167,-0.155,2.67,md,13.0,137.0,0.03243,0.100,...,2016-12-15T05:08:54.636Z,Central California,quarry blast,0.34,1.060,0.77,3.0,reviewed,nc,nc
3,1975-12-30 23:04:47.424,36.838667,-121.569833,4.774,2.67,md,46.0,81.0,0.01892,0.120,...,2016-12-15T05:08:50.998Z,Central California,earthquake,0.21,0.360,0.28,11.0,reviewed,nc,nc
4,1975-12-30 18:10:28.128,35.781000,-121.421000,11.895,3.15,md,17.0,247.0,0.23960,0.090,...,2016-12-15T05:08:39.866Z,offshore Central California,earthquake,1.04,0.610,0.55,4.0,reviewed,nc,nc


In [49]:
proper_df.to_csv('Database/export.csv')